# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,9.93,69,21,0.45,US,1707516720
1,1,edinburgh of the seven seas,-37.0676,-12.3116,19.71,93,99,10.68,SH,1707516890
2,2,blackmans bay,-43.0167,147.3167,13.18,57,27,0.45,AU,1707516889
3,3,albany,42.6001,-73.9662,10.25,59,21,0.45,US,1707516741
4,4,ust'-kut,56.7938,105.7672,-9.76,96,100,2.24,RU,1707517402


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit the criteria and drop any results with null values
    # A max temperature lower than 27 degrees but higher than 21
    # Wind speed less than 4.5 m/s
    # Zero cloudiness

city_data_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)
].copy()

# Drop any rows with null values
city_data_df.dropna(axis='rows', inplace=True)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,saint-pierre,-21.3393,55.4781,25.82,78,0,3.09,RE,1707516946
158,158,port augusta,-32.5000,137.7667,24.02,35,0,4.24,AU,1707516984
176,176,geraldton,-28.7667,114.6000,23.23,23,0,1.54,AU,1707516980
263,263,maun,-19.9833,23.4167,25.98,61,0,1.54,BW,1707517443
323,323,kadugli,11.0167,29.7167,25.04,18,0,3.33,SD,1707517453
331,331,valparaiso,-33.0393,-71.6273,25.13,51,0,4.12,CL,1707517454
371,371,arkadelphia,34.1209,-93.0538,22.54,78,0,4.12,US,1707517461
472,472,bahia honda,22.9036,-83.1592,24.84,54,0,4.04,CU,1707517478
498,498,aioun,16.6614,-9.6149,25.19,9,0,3.93,MR,1707516911
569,569,tabou,4.4230,-7.3528,25.98,89,0,1.36,CI,1707516986


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,saint-pierre,RE,-21.3393,55.4781,78,
158,port augusta,AU,-32.5000,137.7667,35,
176,geraldton,AU,-28.7667,114.6000,23,
263,maun,BW,-19.9833,23.4167,61,
323,kadugli,SD,11.0167,29.7167,18,
331,valparaiso,CL,-33.0393,-71.6273,51,
371,arkadelphia,US,34.1209,-93.0538,78,
472,bahia honda,CU,22.9036,-83.1592,54,
498,aioun,MR,16.6614,-9.6149,9,
569,tabou,CI,4.4230,-7.3528,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
port augusta - nearest hotel: Hotel Commonwealth
geraldton - nearest hotel: Ocean Centre Hotel
maun - nearest hotel: Center Lodge Conference Center
kadugli - nearest hotel: South Kordofan Hotel
valparaiso - nearest hotel: Ibis Hotel
arkadelphia - nearest hotel: Captain Henderson House
bahia honda - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
tabou - nearest hotel: hôtel le rochet


,City,Country,Lat,Lng,Humidity,Hotel Name
20,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
158,port augusta,AU,-32.5000,137.7667,35,Hotel Commonwealth
176,geraldton,AU,-28.7667,114.6000,23,Ocean Centre Hotel
263,maun,BW,-19.9833,23.4167,61,Center Lodge Conference Center
323,kadugli,SD,11.0167,29.7167,18,South Kordofan Hotel
331,valparaiso,CL,-33.0393,-71.6273,51,Ibis Hotel
371,arkadelphia,US,34.1209,-93.0538,78,Captain Henderson House
472,bahia honda,CU,22.9036,-83.1592,54,No hotel found
498,aioun,MR,16.6614,-9.6149,9,فندق العيون
569,tabou,CI,4.4230,-7.3528,89,hôtel le rochet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)